# Chatbot to help in communication between refugees and personnel

Here I am importing the needed modules

In [1]:
import nltk
import io
import numpy as np
import random
import string
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")

Opening the file with the info for chatbot to give.

In [2]:
f = open('details.txt')
raw = f.read().lower()
nltk.download("punkt")
nltk.download("wordnet")
sent_tokens = nltk.sent_tokenize(raw) #Puts each sentence of paragraph into a list/array
word_tokens = nltk.word_tokenize(raw) #Puts each word of a sentence into a list/array


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
sent_tokens[:2]

['kakuma is a town in northwestern turkana county, kenya.',
 'it is the site of a unhcr refugee camp, established in 1992.']

In [4]:
word_tokens[:2]

['kakuma', 'is']

### Pre-processing

In [5]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct)))



### Greetings database

In [6]:
USER_GREETINGS = ('hello', 'hi', 'hey', 'greetings', 'good morning')
BOT_GREETINGS = ["hi", "hey", "hello", "nice to meet you", "hi what's up"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in USER_GREETINGS:
            return random.choice(BOT_GREETINGS)

In [7]:
def respond(user_input):
    chatbot_response = ''
    sent_tokens.append(user_input)
    Vec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
    tfidf = Vec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if req_tfidf == 0:
        chatbot_response = chatbot_response + """Sorry I don't understand"""
        return chatbot_response
    else:
        chatbot_response = chatbot_response + sent_tokens[idx]
        return chatbot_response
    

In [11]:
running = True
print("Hello I am Team 1's chatbot version 0.1, type 'bye' to exit")

reports = list()

while running:
    print("")
    print("""Push '1' or 'report' to file a complaint.""")
    print("""Push '2' or 'info' to get info on the camp.""")
    print("""Push '3' or 'survey' to fill in the latest survey""")
    user_input = input().lower()
    if user_input != "bye":
        if user_input == "thanks" or user_input == "thank you":
            running = False
            print("""Bot: You're welcome""")
            
        elif user_input == "1" or user_input == "report":
            myreport = input("Please describe the issue: ")
            reports.append(myreport)
            print("Thank you for the feedback. The authorities will be informed that: %s and will be acted upon" %myreport)
        elif user_input == "2" or user_input == "info":
            question = input("Please ask anything about the camp: ").lower()
            if greeting(question) != None:
                print("Chatbot: "+ greeting(user_input))
            else:
                print("Chatbot: ", end="")
                print(respond(question))
                sent_tokens.remove(question)
        elif user_input == "3" or user_input == "survey":
            print("Work in Progress")
        else:
            if greeting(user_input) != None:
                print("Chatbot: "+ greeting(user_input))
            else:
                print("Chatbot: ", end="")
                print(respond(user_input))
                sent_tokens.remove(user_input)
    else:
        running = False
        print("Chatbot: Bye, have a nice day.")
    
    
    

Hello I am Team 1's chatbot version 0.1, type 'bye' to exit

Push '1' or 'report' to file a complaint.
Push '2' or 'info' to get info on the camp.
Push '3' or 'survey' to fill in the latest survey
1
Please describe the issue: No water
Thank you for the feedback. The authorities will be informed that: No water and will be acted upon

Push '1' or 'report' to file a complaint.
Push '2' or 'info' to get info on the camp.
Push '3' or 'survey' to fill in the latest survey
2
Please ask anything about the camp: how many people are in kakuma
Chatbot: [2] many people in kakuma are long-term refugees, living in hopelessness and desperation.

Push '1' or 'report' to file a complaint.
Push '2' or 'info' to get info on the camp.
Push '3' or 'survey' to fill in the latest survey
3
Work in Progress

Push '1' or 'report' to file a complaint.
Push '2' or 'info' to get info on the camp.
Push '3' or 'survey' to fill in the latest survey
bye
Chatbot: Bye, have a nice day.
